## Step 9 - Run the application on the Production Centre

Step 9 of the tutorial uses a Jupyter Notebook to query the catalog for a Sentinel-1 GRD product, creates a Web Processing Service (WPS) request invoking the data transformation application that was deployed in **Step 8**, monitors the WPS request execution and finally retrieves the data transformation execution results

* First do the imports of the Python libraries required

In [1]:
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService
import sys
import os
sys.path.append('/opt/anaconda/bin/')
import lxml.etree as etree
os.environ["TMPDIR"] = '1'
os.environ["_CIOP_APPLICATION_PATH"] = '1'
import requests
import cioppy
ciop = cioppy.Cioppy()
import shapely
import dateutil.parser
from shapely.wkt import loads 
import pandas as pd
import geopandas as gp
from datetime import datetime, timedelta
from io import BytesIO
from zipfile import ZipFile
%matplotlib inline
import matplotlib.pyplot as plt
import gdal
import numpy as np

* Define the search parameters: the catalog series OpenSearch endpoint, the time of interest and the area of interest

In [2]:
series = 'https://catalog.terradue.com/sentinel1/search'

start_date = '2017-09-01T00:00:00'
stop_date = '2017-12-10T23:59:59'

geom = 'MULTIPOLYGON (((26.832 9.5136, 28.6843 9.5136, 28.6843 7.8009, 26.832 7.8009, 26.832 9.5136)), ((32.0572 12.4549, 33.9087 12.4549, 33.9087 10.7344, 32.0572 10.7344, 32.0572 12.4549)), ((-5.5 17.26, -1.08 17.26, -1.08 13.5, -5.5 13.5, -5.5 17.26)), ((12.9415 13.7579, 14.6731 13.7579, 14.6731 12.0093, 12.9415 12.0093, 12.9415 13.7579)))'

* Search for Sentinel-1 GRD products in one of the polygons of the area of interest

In [ ]:
wkt = loads(geom)[0].wkt

In [5]:
search_params = dict([('geom', wkt),
                     ('start', start_date),
                     ('stop', stop_date),
                      ('pt', 'GRD')])

In [6]:
search = ciop.search(end_point = series,
                     params = search_params,
                     output_fields='self,enclosure,identifier', 
                     model='GeoTime')

* List the Sentinel-1 GRD products found

In [7]:
for index, elem in enumerate(search):
    print(index, elem['identifier'])

(0, 'S1A_IW_GRDH_1SDV_20171210T182114_20171210T182139_019644_021603_0AB1')
(1, 'S1A_IW_GRDH_1SDV_20171210T182049_20171210T182114_019644_021603_96BE')
(2, 'S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33')
(3, 'S1A_IW_GRDH_1SDV_20171210T181959_20171210T182024_019644_021603_D3D2')
(4, 'S1A_IW_GRDH_1SDV_20171209T033328_20171209T033353_019620_02154C_F3B6')
(5, 'S1A_IW_GRDH_1SDV_20171209T033259_20171209T033328_019620_02154C_8F85')
(6, 'S1A_IW_GRDH_1SDV_20171208T183711_20171208T183736_019615_02151E_1BB2')
(7, 'S1A_IW_GRDH_1SDV_20171208T183646_20171208T183711_019615_02151E_561D')
(8, 'S1A_IW_GRDH_1SDV_20171208T183621_20171208T183646_019615_02151E_6721')
(9, 'S1B_IW_GRDH_1SDV_20171208T034043_20171208T034108_008622_00F4FA_EBD2')
(10, 'S1B_IW_GRDH_1SDV_20171208T034018_20171208T034043_008622_00F4FA_796B')
(11, 'S1A_IW_GRDH_1SDV_20171207T035045_20171207T035110_019591_021463_A5A3')
(12, 'S1A_IW_GRDH_1SDV_20171207T035016_20171207T035045_019591_021463_0026')
(13, 'S1A_IW_GRDH_1SDV

* Select the Sentinel-1 GRD product to process

In [8]:
s1_index = 2

* Connect to the WPS server

In [43]:
wps_url = 'https://ec-better-apps-deployer.terradue.com/zoo-bin/zoo_loader.cgi'

wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

* Do a GetCapabilities WPS request and list the process:

In [44]:
wps.getcapabilities()

In [45]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'UnDeployProcess')
(1, 'ec_better_wfp_01_01_01_wfp_01_01_01_0_1')
(2, 'DeployProcess')
(3, 'GetStatus')


* Select the **ec_better_wfp_01_01_01_wfp_01_01_01_0_1** process and print the title and abstract after having submited a WPS DescribeProcess request

In [ ]:
process_id = wps.processes[1].identifier

In [ ]:
process = wps.describeprocess(process_id)

In [ ]:
process.title

In [17]:
process.abstract

'WFP-01-01-01 Data transformation application - Sentinel-1 backscatter timeseries'

* List the WPS process inputs:

In [18]:
for input in process.dataInputs:
    print(input.identifier)

source
filterSizeX
filterSizeY
polarisation
wkt
quotation
_T2Username


* Create a Python dictionary with the inputs:

In [ ]:
filterSizeX = '5'
filterSizeY = '5'
polarisation = 'VV'

In [ ]:
inputs = [('source', search[s1_index]['self']),
        ('filterSizeX', filterSizeX),
        ('filterSizeY', filterSizeY),
        ('polarisation', polarisation),
        ('wkt', wkt),
        ('quotation', 'No'),
        ('_T2Username', 'fbrito')]

* Submit the Execute WPS request:

In [25]:
execution = owslib.wps.WPSExecution(url=wps.url)

execution_request = execution.buildRequest(process_id, 
                                           inputs, 
                                           output = [('result_osd', False)])

execution_response = execution.submitRequest(etree.tostring(execution_request))

execution.parseResponse(execution_response)

* Monitor the request:

In [26]:
execution.statusLocation

'http://ec-better-apps-deployer.terradue.com/zoo-bin/zoo_loader.cgi?request=Execute&service=WPS&version=1.0.0&Identifier=GetStatus&DataInputs=sid=0f3e49a2-746d-11e8-9801-0242ac11000f&RawDataOutput=Result'

In [27]:
monitorExecution(execution)

 owslib.wps.WPSException : {'locator': 'https://recast.terradue.com/t2api/searchfile/fbrito/_results/workflows/ec_better_wfp_01_01_01_wfp_01_01_01_0_1/run/0f3e49a2-746d-11e8-9801-0242ac11000f/0004993-171122163343932-oozie-oozi-W/.t2error.json', 'code': 'NoApplicableCode', 'text': 'Task KILL is received. Killing attempt!\n'}


* Check the outcome of the processing request

In [28]:
execution.isSucceded()

False

* Search for the results produced

In [29]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


In [30]:
results_osd = execution.processOutputs[0].reference

In [33]:
results_osd

In [32]:
search_results = ciop.search(end_point=results_osd,
                         params=[],
                         output_fields='identifier,enclosure',
                         model='GeoTime')

TypeError: execv() arg 2 must contain only strings

In [ ]:
for index, elem in enumerate(search_results):
    print(elem['enclosure'])